In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import geopandas as gpd
import requests
from tqdm.notebook import tqdm

from scripts import deploy_contours

In [ ]:
# User inputs
model_version = '0.0.11'
model_name = 'spectrogram_v0.0.11_2021-07-13'
model_file = '../models/' + model_name + '.h5'

version_num = str(2.1)
contour_output_path = f'../data/model_outputs/site_contours/{version_num}'
if not os.path.exists(contour_output_path):
    os.mkdir(contour_output_path)

# Rect needs to be large enough to cover big sites. 
# But large rects take longer to process and increase false positive likelihood
rect_width = 0.01
mosaic_period = 2
spectrogram_interval = 3
mosaic_method = 'min'
scale=4

# Note on dates: The date range should be longer than the spectrogram length.
# Starting on successive mosaic periods (typically: monthly), as many
# spectrograms are created as fit in the date range.
start_date = '2015-12-01'
end_date = '2022-03-31'

confirmed_sites_file = 'compiled_sites'
confirmed_sites = gpd.read_file(f"../data/site_metadata/{confirmed_sites_file}.geojson")

product_id = f'earthrise:contours_{confirmed_sites_file}_v{version_num}_{start_date}_{end_date}_{mosaic_period}_{spectrogram_interval}'
product_name = product_id.split(':')[-1]  # Arbitrary string - optionally set this to something more human readable.

endpoint = 'http://api.plastic.watch.earthrise.media'
run_local = False # If False, the model prediction tasks are async queued and sent to DL for processing.

In [ ]:
args = [
    '--file_name',
    confirmed_sites_file,
    '--product_id',
    product_id,
    '--product_name',
    product_name,
    '--model_file',
    model_file,
    '--model_name',
    model_name,
    '--mosaic_period',
    str(mosaic_period),
    '--mosaic_method',
    mosaic_method,
    '--spectrogram_interval',
    str(spectrogram_interval),
    '--start_date',
    start_date,
    '--end_date',
    end_date,
    '--rect_width',
    str(rect_width),
    '--endpoint',
    endpoint
]
if run_local:
    args.append('--run_local')

In [ ]:
# Because of the way DL uploads modules when queuing async tasks, we need to launch from the scripts/ folder
%cd ../scripts
%pwd

In [ ]:
deploy_contours.main(args)

In [ ]:
contours = gpd.GeoDataFrame()
for site_id in tqdm(confirmed_sites['id']):
    site_endpoint = f'{endpoint}/sites/{site_id}/contours'
    r = requests.get(site_endpoint).json()
    contours = contours.append(gpd.GeoDataFrame.from_features(r['features']))
    file_path = f'{contour_output_path}/{confirmed_sites_file}_{start_date}_{end_date}_period_{mosaic_period}_interval_{spectrogram_interval}_contours.geojson'
    contours.to_file(file_path, driver='GeoJSON')